In [6]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer


In [2]:
df = pd.read_csv("./data/credit_npo.csv")

In [3]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12537 entries, 0 to 12536
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   SeriousDlqin2yrs                      12537 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  12537 non-null  float64
 2   age                                   12537 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  12537 non-null  int64  
 4   DebtRatio                             12537 non-null  float64
 5   MonthlyIncome                         11816 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       12537 non-null  int64  
 7   NumberOfTimes90DaysLate               12537 non-null  int64  
 8   NumberRealEstateLoansOrLines          12537 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  12537 non-null  int64  
 10  NumberOfDependents                    12360 non-null  float64
dtypes: float64(4), 

In [4]:
X = df.drop(columns=['SeriousDlqin2yrs'])
y = df['SeriousDlqin2yrs']

In [7]:
# Creación de un pipeline con imputación para manejar NaNs
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar valores faltantes con la media
    ('scaler', StandardScaler()),                 # Normalización
    ('pca', PCA()),                               # Reducir dimensionalidad si es necesario
    ('kmeans', KMeans())                          # Clustering
])

In [8]:
# Configuración de GridSearchCV y Validación Cruzada
param_grid = {
    'pca__n_components': [2, 3, 5],               # Intentar diferentes dimensiones
    'kmeans__n_clusters': [2, 3, 4, 5, 6, 7, 8],  # Número de clusters
    'kmeans__init': ['k-means++', 'random']
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('pca', PCA()), ('kmeans', KMeans())]),
             param_grid={'kmeans__init': ['k-means++', 'random'],
                         'kmeans__n_clusters': [2, 3, 4, 5, 6, 7, 8],
                         'pca__n_components': [2, 3, 5]})

In [9]:
# Resultados de GridSearchCV y evaluación
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor score de validación:", grid_search.best_score_)

Mejores parámetros: {'kmeans__init': 'random', 'kmeans__n_clusters': 8, 'pca__n_components': 2}
Mejor score de validación: -431.06215034299083


In [10]:
# Validación cruzada con los mejores parámetros
best_model = grid_search.best_estimator_
scores = cross_val_score(best_model, X, y, cv=5)
print("Puntajes de validación cruzada:", scores)
print("Puntaje promedio de validación cruzada:", scores.mean())

Puntajes de validación cruzada: [-328.47379383 -656.8235411  -449.18652766 -345.45674405 -383.84446783]
Puntaje promedio de validación cruzada: -432.75701489240635


In [11]:
# Regresión y análisis del cluster asignado frente al objetivo
# Obtenemos los clusters usando el mejor modelo
best_model.fit(X)
clusters = best_model.named_steps['kmeans'].labels_

In [12]:
# Crear un nuevo DataFrame con la variable de cluster y el target
data_clustered = X.copy()
data_clustered['cluster'] = clusters
data_clustered['target'] = y

In [13]:
# Realizar regresión entre el cluster y el target
X_clustered = data_clustered[['cluster']]
reg = LinearRegression()
reg.fit(X_clustered, y)
y_pred = reg.predict(X_clustered)

In [14]:
# Calcular error cuadrático medio
mse = mean_squared_error(y, y_pred)
print("Error cuadrático medio de la regresión:", mse)

Error cuadrático medio de la regresión: 0.06405726964057512
